In [2]:
import requests
from bs4 import BeautifulSoup

url = "https://app.yamol.tw/exam/24548"
resonpse = requests.get(url)
soup = BeautifulSoup(resonpse.text,"html.parser")
print(soup.prettify()) # soup.prettify() 會將 html 格式化

<!DOCTYPE html>
<html lang="zh-Hant-TW">
 <head>
  <meta charset="utf-8"/>
  <meta content="no-cache" http-equiv="Pragma"/>
  <meta content="no-cache, no-store, must-revalidate" http-equiv="cache-control"/>
  <link href="/favicon.ico" rel="shortcut icon"/>
  <meta content="width=device-width,initial-scale=1" name="viewport"/>
  <meta content="#000000" name="theme-color"/>
  <meta content="èå¬è·ãæçãè­ç§ãåèãç¹èï¼ææç¨çé¡åº«æ¸¬é©ç³»çµ±ï¼æ¯æ¨ä¸æ¦çå¿åå·¥å·!" name="description"/>
  <meta content="é¿æ©ç·ä¸æ¸¬é© - é¯å¨é¿æ©ï¼è´å¨èå ´ï¼" property="og:title">
   <meta content="èå¬è·ãæçãè­ç§ãåèãç¹èï¼ææç¨çé¡åº«æ¸¬é©ç³»çµ±ï¼æ¯æ¨ä¸æ¦çå¿åå·¥å·!" property="og:description"/>
   <meta content="https://yamol.tw/images/logo_new.jpg" property="og:image"/>
   <meta content="website" property="og:type"/>
   <script async="" src="https://www.googletagmanager.com/gtag/js?id=G-YLQXFP87X7">
   </script>
   <l

In [3]:
# 靜態爬蟲無法爬蟲以下網頁的問題內容，因此使用動態爬蟲。
question_div = soup.select_one('div[class*="MuiBox-root"]')

if question_div:
    question_content = question_div.get_text(strip=True)
    print(question_content)
else:
    print("指定的 div 標籤未找到。")

指定的 div 標籤未找到。


In [4]:
# from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium import webdriver
import time
import pandas as pd
import re

# 登入帳號密碼
username = 'your_username'
password = 'your_password'

# 設置 Chrome 瀏覽器選項
chrome_options = Options()
chrome_options.add_argument("--headless")  # 啟用無頭模式
chrome_options.add_argument("--disable-gpu")

# 設置 ChromeDriver 的路徑
driver_path = 'chromedriver.exe'  # 替換為你的 chromedriver 路徑
service = Service(driver_path)

# 初始化 WebDriver
driver = webdriver.Chrome(service=service, options=chrome_options)

# 載入網頁
url = 'https://app.yamol.tw/exam/76590'  # 替換為你的目標 URL
driver.get(url)

# 等待網頁加載完成
time.sleep(5)  # 視情況調整等待時間

question_content = []  # 建立空的 list 儲存題目內容
options_content = []  # 建立空的 list 儲存選項內容
answer_content = []  # 建立空的 list 儲存答案內容
new_url_content = []  # 建立空的 list 儲存新 URL

visited_questions = set()  # 用來儲存已經抓取過的題目


# 點擊Index切換網頁內容
i = 1
while True:
    try:
        button = driver.find_element(By.XPATH, f"//button[contains(@class, 'text-info') and text()='{i}. ']")
        print(f"點擊 {i}. 題")
        i += 1
        button.click()
    except:
        print("已完成所有頁面")
        break

    # 等待新頁面加載完成
    time.sleep(2)

    # 用By.XPATH 定位到 QUESTION 1 並抓取內容
    stat_div = driver.find_element(By.XPATH, "//div[contains(text(), '統計：')]")
    statistics_text = stat_div.text
    print("統計內容：", statistics_text)

    match = re.search(r'#(\d+)', statistics_text)
    if match:
        seiunm = match.group(1)
        new_url = f"{url}?info=item.{seiunm}"  # 取得新的URL
        new_url_content.append(new_url)
        print("新的 URL:", new_url)

        if seiunm in visited_questions:
            print("已經抓取過該題目")
            continue

        visited_questions.add(seiunm)

        # 點擊另一個題目頁面
        driver.get(new_url)
        time.sleep(2)

        # 定位到 題目欄位 並抓取內容
        question_div = driver.find_element(By.CLASS_NAME, 'item-htmlRender')
        inner_divs = question_div.find_element(By.CLASS_NAME, 'MuiBox-root')
        question_content.append(inner_divs.text)

        # 定位到 選項欄位 並抓取內容
        options_div = driver.find_element(By.ID, f'item-{seiunm}-options')
        inner_divs = options_div.find_elements(By.CLASS_NAME, 'MuiBox-root')
        options = [div.text for div in inner_divs]
        options_content.append(options)

        # 定位到 答案欄位 並抓取內容
        try:
            answer_div = driver.find_element(By.XPATH, "//div[contains(text(), '答案：')]/a")
            answer_letter = answer_div.text
        except:
            answer_letter = "登入後查看"  # 如果答案無法取得
        answer_content.append(answer_letter)

        print(f"問題: {question_content[-1]}")
        print(f"選項: {options}")
        print(f"答案: {answer_letter}")

# 將題目、選項、答案加入 DataFrame
df = pd.DataFrame(columns=['題目', 'A', 'B', 'C', 'D', 'E', '答案', '題目代碼'])

for i in range(len(question_content)):
    options = options_content[i] + [""] * (5 - len(options_content[i]))  # 補足選項至5個
    df.loc[i] = [
        question_content[i],
        options[0],
        options[1],
        options[2],
        options[3],
        options[4],
        answer_content[i],
        new_url_content[i]
    ]

df.to_csv('python.csv', index=False) # 儲存成 CSV 檔案

# 關閉瀏覽器
driver.quit()
print(df)


NoSuchElementException: Message: no such element: Unable to locate element: {"method":"xpath","selector":"//a[contains(@class, 'MuiButton-containedPrimary') and contains(text(), '登入')]"}
  (Session info: chrome=125.0.6422.61); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
	GetHandleVerifier [0x00007FF6663F22C2+60002]
	(No symbol) [0x00007FF66636CA59]
	(No symbol) [0x00007FF666227EDA]
	(No symbol) [0x00007FF6662776E6]
	(No symbol) [0x00007FF6662777AC]
	(No symbol) [0x00007FF6662BE9D7]
	(No symbol) [0x00007FF66629C2CF]
	(No symbol) [0x00007FF6662BBCC7]
	(No symbol) [0x00007FF66629C033]
	(No symbol) [0x00007FF666269657]
	(No symbol) [0x00007FF66626A251]
	GetHandleVerifier [0x00007FF666703E2D+3278285]
	GetHandleVerifier [0x00007FF666750190+3590448]
	GetHandleVerifier [0x00007FF6667461D0+3549552]
	GetHandleVerifier [0x00007FF6664A1DE6+779654]
	(No symbol) [0x00007FF666377ACF]
	(No 